In [19]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수 사용
SOLAR_API_KEY = os.getenv('SOLAR_API_KEY')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
GPT_API_KEY = os.getenv('GPT_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.getenv('GEMINI_API_KEY')

In [20]:
from openai import OpenAI # openai==1.2.0
from groq import Groq

In [21]:
def get_solar_completion(input_text, system_prompt):
    client = OpenAI(
        api_key=SOLAR_API_KEY,
        base_url="https://api.upstage.ai/v1/solar"
    )

    stream = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=[
          {
            "role": "system",
            "content": system_prompt
          },
          {
            "role": "user",
            "content": input_text
          }
        ],
        stream=False,
    )
    return stream.choices[0].message.content,

In [23]:
def get_groq_completion(input_text, system_prompt, model="mixtral-8x7b-32768"):
    client = Groq(
        api_key=GROQ_API_KEY,
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": input_text,
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content, 

In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

def get_gemini_completion(input_text, system_prompt):
    model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True, )
    message = model(
        [
            SystemMessage(content=system_prompt),
            HumanMessage(content=input_text),
        ]
    )
    return message.content

In [25]:
def get_gpt_completion(input_text, system_prompt):
    client = OpenAI(api_key=GPT_API_KEY)

    completion = client.chat.completions.create(
      model="gpt-3.5-turbo-0125", # gpt-4-0125-preview, gpt-3.5-turbo-0125
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": input_text}
      ]
    )
    return completion.choices[0].message

In [26]:
def get_completion(input_text, system_prompt):
    solar_completion = get_solar_completion(input_text, system_prompt)
    mixtral_groq_completion = get_groq_completion(input_text, system_prompt, model="mixtral-8x7b-32768")
    llama2_groq_completion = get_groq_completion(input_text, system_prompt, model="llama2-70b-4096")
    gemini_completion = get_gemini_completion(input_text, system_prompt)
    #gpt_completion = get_gpt_completion(input_text, system_prompt)
    
    messages = [
        solar_completion,
        mixtral_groq_completion,
        llama2_groq_completion,
        gemini_completion,
        #gpt_completion
    ]

    return messages

In [27]:
system_prompt = "You are a helpful assistant."
input_text = "안녕"

In [28]:
messages = get_completion(input_text, system_prompt)
messages

In [34]:
for i, message in enumerate(messages):
    print(i, message)

0 ('안녕하세요! 도움이 필요하신가요?',)
1 ("안녕하세요! 제가 도움이 필요한 대 part of 수 있도록 최선을 다하겠습니다. (Hello! I'll do my best to be of help.) How can I assist you today?",)
2 ("Hello! I'm here to assist you in any way I can. Is there something specific you need help with or a task you'd like to accomplish? Let me know and I'll do my best to aid you.",)
3 안녕하세요! 도움이 필요하신가요?
